In [3]:
import os
from glob import glob
import pandas as pd
import html
import itertools
import pickle
import re
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.naive_bayes import MultinomialNB,GaussianNB
from sklearn.cluster import KMeans, k_means_, DBSCAN, AgglomerativeClustering
from sklearn import cluster
from sklearn.metrics import silhouette_score
from sklearn.metrics.pairwise import cosine_distances

import hdbscan
from gensim import models
from gensim.corpora import mmcorpus, Dictionary
from gensim.models import lsimodel, ldamodel, tfidfmodel, rpmodel, logentropy_model, TfidfModel, LsiModel, LdaModel
from gensim import matutils, corpora
from gensim.models.doc2vec import TaggedDocument
from gensim.models import doc2vec
from gensim.models.word2vec import Word2Vec
from sklearn.svm import SVC

import sys
sys.path.append('~/Documents/GitHub/Private_Project/personal_project/')
import html
from collections import Counter
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import chat_bot as cb
import Database_Handler as dh
from datetime import datetime
import seaborn as sns
import functools
from konlpy.tag import Mecab
from konlpy.utils import pprint
mecab = Mecab()
%matplotlib inline

In [8]:
dataDict = pickle.load(open('./data/pre_data/for_statistics_daum_from_mongodb.pickled','rb'))
print (len(dataDict))

4722


In [10]:
keywordsDict = pickle.load(open('./data/pre_data/keywords_daum.pickled','rb'))

In [12]:
for idx in dataDict:
    dataDict[idx]['extracted_keywords'] = keywordsDict[idx]

In [13]:
df_daum = pd.DataFrame.from_dict(dataDict, orient='index')
df_daum['date'] = pd.to_datetime(df_daum['date']).dt.date
df_daum.reset_index(inplace = True)
df_daum.rename(columns={'index':'id'}, inplace=True)
print (df_daum.shape)
df_daum.head()

(4722, 11)


,id,category,date,press,number_of_comment,number_of_crawled_comment,rank,title,mainText,keywords,extracted_keywords
0,5a2a61bf588c13481c229d1e,뉴스,2017-12-07,세계일보,1093,911,1,"""밤이 무섭다""..비아그라 공장 연기에 남성들 부작용 호소","주민들은 공장에서 배출된 연기가 '남성이 매우 건강해지는 부작용'을 일으킨다며, ...","[부작용, 비아그라, 아일랜드]","{건강, 부작용, 세보 효과, 지역, 공장, 남성들, 연기}"
1,5a2a61bf588c13481c229d1f,뉴스,2017-12-07,헬스조선,603,386,2,식후 커피·늦은 양치질..점심식사 후 하면 안 좋은 습관 3가지,점심식사를 마친 후 후식으로 커피를 마시는 사람들이 많다. 실제로 직장이 밀집돼 ...,"[커피, 낮잠, 음식물]","{건강, 점심 식사, 철분, 자세, 입냄새, 커피, 디스크, 식후, 치아, 낮잠}"
2,5a2a61bf588c13481c229d20,뉴스,2017-12-07,연합뉴스,1067,811,3,"'십년지기 생매장' 진짜 이유는..""'청부 통정' 알려질까 봐""",(성남=연합뉴스) 최해민 기자 = 십년지기 지인을 산 채로 묻어 살해한 50대 여...,"[살인혐의, 철원, 검찰송치]","{앙심, 남편, 성관계, 진술, 경찰, 아들, 범행, 주변, 철원, 지인}"
3,5a2a61bf588c13481c229d21,뉴스,2017-12-07,헤럴드경제,418,369,4,"신영자, 억 소리나는 갑질","신영자, 적용안된 혐의→검찰 상고에서 인정\n신영자, 얼마를 어떻게 받았나 [헤럴...","[신영자, 갑질, 롯데백화점]","{유통업체, 롯데, 매장, 네이처리퍼블릭, 검찰, 신영자 이사장, 혐의, 징역}"
4,5a2a61bf588c13481c229d22,뉴스,2017-12-07,연합뉴스,434,368,5,"""배신하지마"" 20대女 살인 피의자 유치장서 공범 남친에 쪽지",(청주=연합뉴스) 이승민 기자 = 지난 9월 청주의 한 하천에서 20대 여성을 둔기...,"[공범, 살인, 과자]","{쪽지, 경찰, 폭행, 유치장, 과자, 남자친구, 범행, 혐의}"


In [14]:
textInfo = df_daum.title.values + df_daum.mainText.values

In [15]:
def nav_tokenizer(corpus):
    pos = mecab.pos(corpus)
    res = [x[0] for x in pos if (x[1] == u'VV' or x[1] == u'VA' or x[1] == u'NNB' or x[1] == u'NNP' or x[1] == u'NNG')]
    return res

In [16]:
def remove_stopwords(words, stopwords):
    res = [x for x in words if x not in stopwords]
    return res

In [19]:
stopwords = open('./data/stopwordsList.txt',encoding='utf-8').readlines()
stopwords = list(map(lambda x: x.strip(), stopwords))

In [21]:
d2v_docs = [nav_tokenizer(x) for x in textInfo]
d2v_docs = [remove_stopwords(x, (stopwords)) for x in d2v_docs]

In [22]:
tagged_d2v_docs = [TaggedDocument(doc, [idx]) for idx, doc in enumerate(d2v_docs)]

In [23]:
d2v_model = doc2vec.Doc2Vec(size=500, window=5, alpha=0.025, min_alpha=0.025, seed=0)
d2v_model.build_vocab(tagged_d2v_docs)
d2v_model.train_lbls = False # do not train labels of words
d2v_model.train_words = True # only train relations among words

In [24]:
for epoch in range(30):
    d2v_model.train(tagged_d2v_docs, total_examples=len(d2v_docs), epochs=epoch)
    d2v_model.alpha -= 0.002 # decrease the learning rate
    d2v_model.min_alpha = d2v_model.alpha # fix the learning rate, no decay

In [25]:
X_d2v = [d2v_model.infer_vector(x.words) for x in tagged_d2v_docs]

In [26]:
idx = 5
pprint('text: '+ textInfo[idx])
print("")
for x in d2v_model.docvecs.most_similar(positive=[X_d2v[idx]]):
    pprint(textInfo[x[0]] + ' (%3f)' % x[1])

('text: 화성서 발견된 포탄 모양 둥근 물체, 정체는?(지디넷코리아=이정현 미디어연구소)화성에서 포탄처럼 생긴 둥근 모양의 물체가 '
 '발견됐다. 미국 항공우주국(NASA)은 6일(현지시간) 화성 탐사 로봇 큐리오시티 로버 공식 트위터 계정을 통해 화성에서 발견된 그란 '
 '모양의 물체 사진을 공개했다고 주요 외신들이 보도했다. 공개된 사진 속에는 먼지로 뒤덮인 화성의 지표면에 놓여있는 동그란 모양의 물체가 '
 '눈에 띈다. 마치 포탄처럼 보이기도 한다.  이 사진은 원래 올해 초 큐리오시티 로버가 촬영한 사진이지만 최근 한 UFO 학자의 블로그에 '
 '새롭게 게시돼 화제를 모았다. 이 사진을 올린 UFO 학자는 이 동그란 모양의 물체는 포탄처럼 생겼다며, 이는 화성에서 전쟁이 일어났던 '
 '증거라고 밝혔다. 하지만 NASA는 이 물체의 크기는 실제로 5mm 미만으로 황산칼슘, 나트륨 및 마그네슘으로 구성된 암석이라고 밝히며, '
 '이 학자의 이론은 틀렸다고 밝혔다. NASA는 화성에서 동그랗게 생긴 암석이 발견된 것은 이번이 처음이 아니라며, 2014년 공개됐던 '
 '사진도 함께 공개했다. 이 때 공개된 블루베리를 닮은 동그란 암석의 크기는 약 3cm 정도였다.  당시, 이 동그란 암석의 정체를 두고 '
 '화산 폭발설, 암석 내부에 수분이 결집되면서 생성된 것이라며 화성에 물이 있었다는 증거라는 의견, 운석이 충돌해 생긴 것이라는 설 등 '
 '다양한 주장이 제기되기도 했다. 이정현 미디어연구소(jh7253@zdnet.co.kr)')

('샤이니 종현, 오늘(18일) 숨진 채 발견그룹 샤이니 종현(27)이 18일 숨진 채 발견됐다. 18일 관계자에 따르면 종현은 이날 오후 '
 '서울 청담동 한 오피스텔에서 숨진 채 발견됐다. 문완식 기자 munwansik@<저작권자 ⓒ ‘리얼타임 연예스포츠 속보,스타의 모든 것’ '
 '스타뉴스, 무단전재 및 재배포 금지> (0.944849)')
'5년 만에 출근하는 이용마 MBC 기자[CBS노컷뉴스 이

In [27]:
idx = 121
pprint('text: '+ textInfo[idx])
print("")
for x in d2v_model.docvecs.most_similar(positive=[X_d2v[idx]]):
    pprint(textInfo[x[0]] + ' (%3f)' % x[1])

("text: 텍사스-마이너 계약 조건 공개.. '3년-2800만 달러' 최근 텍사스 레인저스와 자유계약(FA) 선수 자격을 얻은 왼손 투수 "
 '마이크 마이너(30) 사이에 체결된 계약 조건이 공개됐다. 메이저리그 공식 홈페이지 MLB.com은 7일(이하 한국시각) 텍사스와 '
 '마이너의 계약은 3년-2800만 달러라고 전했다. 이는 연평균 933만 달러. 마이너가 선발 투수로 재기한다면, 적은 금액이다. 또한 '
 '선발 투수로 실패한 뒤 불펜 투수로 좋은 모습을 보인다면, 최상급의 대우다. 텍사스는 마이너를 구원 투수가 아닌 선발 투수로 기용할 '
 '전망이다. 이에 이번 시즌 구원으로 재기에 성공한 마이너가 다시 한 번 선발 투수로 거듭날 수 있을지 관심거리다. 마이너는 메이저리그 '
 '6년차의 왼손 투수. 지난 2014년 이후 어깨 수술로 두 시즌 동안 단 1경기에도 나서지 못했다. 이후 마이너는 이번 시즌 캔자스시티 '
 '소속으로 65경기에서 77 2/3이닝을 던지며, 6승 6패 17홀드 6세이브와 평균자책점 2.55 등을 기록했다. 부상이 없다면, 제 '
 '몫을 해줄 수 있는 투수. 지난 2013년에는 애틀란타 브레이브스 소속으로 200이닝을 넘게 던졌다. 텍사스는 마이너에게 2013시즌과 '
 '같은 모습을 기대할 것으로 보인다. 마이너의 두 번째 재기 여부가 주목된다. 동아닷컴 조성운 기자 madduxly@donga.com')

('"중국, 한국 단체관광 다시 금지?"..문체부 "사실 확인 중"【서울=뉴시스】최동준 기자 = 4일 서울 명동 거리에서 외국인 관광객들이 '
 '관광을 하고 있다. 2017.12.04. photocdj@newsis.com 【서울=뉴시스】김정환 기자 = 중국 정부가 한국행 단체관광을 '
 '다시 금지했다는 20일 국내 일부 언론 보도에 대해 문화체육관광부, 한국관광공사 등 관계 기관은 조심스러운 반응을 보이고 있다. 이날 '
 '국내 일부 언론은 중국 베이징과 산둥 지역 국가여유국이 지난 19일(현지시간) 오후

In [28]:
cnum = 50

In [29]:
from sklearn.metrics.pairwise import cosine_distances
from sklearn.cluster import k_means_ 

def euc2cos_distance(X, Y=None, Y_norm_squared=None, squared=False):
    return cosine_distances(X,Y)

k_means_.euclidean_distances = euc2cos_distance

In [30]:
km = KMeans(n_clusters=cnum, init='k-means++', max_iter=300, n_init=20, tol=0.0001)
km.fit(X_d2v)
print("Silhouette Coefficient: %0.3f" % silhouette_score(X_d2v, km.labels_))

Silhouette Coefficient: 0.656


In [31]:
dfRes = pd.DataFrame(textInfo, columns=['text'])
dfRes['label'] = km.labels_

for i in range(cnum):
    print('label{}: {}'.format(i, len(dfRes[dfRes.label == i])))

label0: 3326
label1: 18
label2: 4
label3: 26
label4: 202
label5: 1
label6: 2
label7: 141
label8: 3
label9: 47
label10: 2
label11: 52
label12: 33
label13: 78
label14: 37
label15: 1
label16: 2
label17: 46
label18: 1
label19: 1
label20: 1
label21: 1
label22: 2
label23: 58
label24: 2
label25: 1
label26: 39
label27: 1
label28: 1
label29: 1
label30: 51
label31: 1
label32: 1
label33: 1
label34: 1
label35: 29
label36: 1
label37: 1
label38: 1
label39: 2
label40: 16
label41: 29
label42: 402
label43: 1
label44: 1
label45: 3
label46: 2
label47: 47
label48: 1
label49: 1


In [32]:
# print top keywords
def print_topwords(df, num, bar):
    try:
        temp_cor = df.text.values
        temp_vec = TfidfVectorizer(tokenizer=nav_tokenizer, stop_words=stopwords + mask1)

        temp_vec.fit(temp_cor)

        temp = pd.DataFrame(list(temp_vec.vocabulary_.keys()), columns=['word'])
        temp['idx'] = temp_vec.vocabulary_.values()
        temp.sort_values(by='idx', inplace=True)
        temp['count'] = temp_vec.transform(temp_cor).toarray().sum(axis=0)

        topwords = temp.sort_values(['count'], ascending=False)[:num]    
    
        for i in range(num):
            print(topwords['word'].values[i]),

        if bar == True:
            fig, ax = plt.subplots()
            plt.bar(range(num), topwords['count'].values, align='center')
            plt.xticks([x for x in range(num)], topwords['word'].values)
            ax.set_xticklabels(ax.xaxis.get_majorticklabels(), rotation=90)
            plt.tight_layout()
            plt.show()
        else:
            pass
    except:
        pass

In [33]:
for i in range(cnum):
    print('cluster {}:'.format(i)),
    print_topwords(df=dfRes.loc[dfRes['label'] == i], num=5, bar=False)
    print("")

cluster 0:

cluster 1:

cluster 2:

cluster 3:

cluster 4:

cluster 5:

cluster 6:

cluster 7:

cluster 8:

cluster 9:

cluster 10:

cluster 11:

cluster 12:

cluster 13:

cluster 14:

cluster 15:

cluster 16:

cluster 17:

cluster 18:

cluster 19:

cluster 20:

cluster 21:

cluster 22:

cluster 23:

cluster 24:

cluster 25:

cluster 26:

cluster 27:

cluster 28:

cluster 29:

cluster 30:

cluster 31:

cluster 32:

cluster 33:

cluster 34:

cluster 35:

cluster 36:

cluster 37:

cluster 38:

cluster 39:

cluster 40:

cluster 41:

cluster 42:

cluster 43:

cluster 44:

cluster 45:

cluster 46:

cluster 47:

cluster 48:

cluster 49:



In [34]:
cnum = 50
agg = AgglomerativeClustering(n_clusters=cnum, affinity='cosine', linkage='complete')
agg.fit(X_d2v)
print("Silhouette Coefficient: %0.3f" % silhouette_score(X_d2v, agg.labels_))

Silhouette Coefficient: -0.356


In [35]:
dfRes = pd.DataFrame(textInfo, columns=['text'])
dfRes['label'] = agg.labels_
for i in range(len(dfRes.label.unique())):
    print('label {}: {}'.format(i, len(dfRes.loc[dfRes['label'] == i])))

label 0: 81
label 1: 66
label 2: 55
label 3: 66
label 4: 54
label 5: 78
label 6: 69
label 7: 64
label 8: 64
label 9: 51
label 10: 70
label 11: 62
label 12: 51
label 13: 49
label 14: 54
label 15: 60
label 16: 81
label 17: 60
label 18: 56
label 19: 42
label 20: 44
label 21: 52
label 22: 50
label 23: 54
label 24: 47
label 25: 65
label 26: 34
label 27: 51
label 28: 49
label 29: 55
label 30: 47
label 31: 52
label 32: 39
label 33: 34
label 34: 38
label 35: 2243
label 36: 56
label 37: 36
label 38: 71
label 39: 38
label 40: 41
label 41: 35
label 42: 33
label 43: 31
label 44: 31
label 45: 38
label 46: 28
label 47: 33
label 48: 33
label 49: 31


In [36]:
for i in range(cnum):
    print('cluster {}:'.format(i)),
    print_topwords(df=dfRes.loc[dfRes['label'] == i], num=10, bar=False)
    print("")

cluster 0:

cluster 1:

cluster 2:

cluster 3:

cluster 4:

cluster 5:

cluster 6:

cluster 7:

cluster 8:

cluster 9:

cluster 10:

cluster 11:

cluster 12:

cluster 13:

cluster 14:

cluster 15:

cluster 16:

cluster 17:

cluster 18:

cluster 19:

cluster 20:

cluster 21:

cluster 22:

cluster 23:

cluster 24:

cluster 25:

cluster 26:

cluster 27:

cluster 28:

cluster 29:

cluster 30:

cluster 31:

cluster 32:

cluster 33:

cluster 34:

cluster 35:

cluster 36:

cluster 37:

cluster 38:

cluster 39:

cluster 40:

cluster 41:

cluster 42:

cluster 43:

cluster 44:

cluster 45:

cluster 46:

cluster 47:

cluster 48:

cluster 49:



In [37]:
db = DBSCAN(eps=0.1, min_samples=10, algorithm='brute', metric='cosine')
Y = db.fit_predict(X_d2v)

In [38]:
# Number of clusters in labels, ignoring noise if present.
cnum = len(set(db.labels_)) - (1 if -1 in db.labels_ else 0)

print('Estimated number of clusters: %d' % cnum)
print("Silhouette Coefficient: %0.3f" % silhouette_score(X_d2v, db.labels_))

Estimated number of clusters: 1
Silhouette Coefficient: 0.564


In [39]:

dfRes = pd.DataFrame(textInfo, columns=['text'])
dfRes['label'] = Y

In [40]:
for i in range(len(dfRes.label.unique())):
    if i == 0 :
        print('label -1: {}'.format(len(dfRes.loc[dfRes['label'] == i-1])))
    else:
        print('label {}: {}'.format(i-1, len(dfRes.loc[dfRes['label'] == i-1])))

label -1: 3514
label 0: 1208


In [41]:
for i in range(-1, cnum):
    print('cluster {}:'.format(i)),
    print_topwords(df=dfRes.loc[dfRes['label'] == i], num=5, bar=False)
    print("")

cluster -1:

cluster 0:



In [42]:
lda_docs = [nav_tokenizer(x) for x in textInfo]
lda_docs = [remove_stopwords(x, stopwords) for x in lda_docs]
lda_dict = corpora.Dictionary(lda_docs)
#calulate TF-IDF
tf_ps = [lda_dict.doc2bow(x) for x in lda_docs]
tf_model = models.TfidfModel(tf_ps)
X_lda = tf_model[tf_ps]

In [43]:
# train topic model
ntopics = 50
nwords = 7

np.random.seed(1)
lda_model = models.ldamodel.LdaModel(X_lda, id2word=lda_dict, num_topics=ntopics)
pprint(lda_model.print_topics(num_topics=ntopics, num_words=nwords))

[(0,
  '0.007*"경찰" + 0.007*"데얀" + 0.006*"야산" + 0.006*"아파트" + 0.005*"전주" + '
  '0.004*"강정호" + 0.004*"주민"'),
 (1,
  '0.012*"박나래" + 0.012*"전현무" + 0.011*"기안84" + 0.007*"투표" + 0.005*"이경규" + '
  '0.004*"한혜진" + 0.004*"낚시"'),
 (2,
  '0.005*"연금" + 0.004*"캐나다" + 0.002*"청년" + 0.002*"김현수" + 0.002*"박용택" + '
  '0.002*"당원" + 0.002*"골프"'),
 (3,
  '0.005*"김영철" + 0.003*"자유한국당" + 0.003*"마녀" + 0.003*"홍" + 0.003*"의원" + '
  '0.003*"정려원" + 0.003*"홍준표"'),
 (4,
  '0.004*"김지원" + 0.001*"다희" + 0.001*"북구" + 0.001*"뇌출혈" + 0.001*"중부" + '
  '0.001*"미터" + 0.001*"임정우"'),
 (5,
  '0.006*"강호동" + 0.005*"한화" + 0.005*"코치" + 0.005*"강식당" + 0.005*"추자현" + '
  '0.004*"우효광" + 0.003*"백종원"'),
 (6,
  '0.007*"열애설" + 0.005*"자니윤" + 0.004*"베일" + 0.003*"원투펀치" + 0.003*"이서진" + '
  '0.003*"아구에로" + 0.003*"엄정화"'),
 (7,
  '0.015*"손흥민" + 0.014*"케인" + 0.009*"골" + 0.008*"토트넘" + 0.007*"시티" + '
  '0.005*"프리미어리그" + 0.004*"맨시티"'),
 (8,
  '0.003*"흑기사" + 0.003*"전인지" + 0.003*"정관용" + 0.003*"교회" + 0.002*"신세경" + '
  '0.002*"담합" + 0.002*"발화"'),
 (9,
  '0.010

In [44]:

idx = 11
pprint(textInfo[idx])

("'탈의실 몰카' 혐의 전현직 국가대표 수영선수 5명 전원 무죄 (수원=뉴스1) 최대호 기자 = 여자 수영선수 탈의실에 몰래 카메라를 "
 '설치해 촬영한 혐의로 재판에 넘겨진 전·현직 국가대표 수영선수 5명이 모두 무죄를 선고 받았다. 범죄의 증명이 부족하다는 이유에서다. '
 '이들을 기소한 검찰은 판결문 검토 후 항소여부를 결정한다는 방침이다. 수원지법 형사9단독 반정모 판사는 7일 성폭력 범죄의 처벌 및 '
 '피해자보호 등에 관한 법률 위반(카메라 등 이용촬영) 혐의로 불구속 기소된 A씨(24)와 B씨(25) 등 전·현직 국가대표 수영선수 '
 '5명에 대해 무죄를 선고했다. A씨는 2009~2013년 경기지역 한 체고 수영장 여자 탈의실과 충북의 한 선수촌 여자 수영선수 탈의에 '
 '만년필 형태의 몰래 카메라를 놓아두고 6차례에 걸쳐 선수들의 탈의 장면을 촬영한 혐의로 기소됐다. B씨 등 4명은 A씨의 몰래 카메라 '
 '설치를 돕거나 망을 보는 등 범행에 가담한 혐의로 기소됐다. 반 판사는 "피고인 A씨는 B씨 등 4명의 피고인이 자신의 범행에 가담했다고 '
 '진술하나 B씨 등 4명 모두 일관되게 혐의를 부인하고 있다"며 "당시 여러가지 처한 상황을 볼때 B씨 등이 A씨 범행에 가담할 이유가 '
 '없었던 것으로 판단된다"고 판시했다. 이어 "검사는 A씨의 진술에 기초해 B씨 등 4명을 기소했는데 유죄를 인정하려면 그 인과관계가 '
 '합리적인 의심의 여지가 없을 정도로 증명이 돼야 하는 데 그렇지 못하다"며 무죄를 선고했다. 반 판사는 A씨에 대해서도 "피고인은 자신이 '
 '범행했다고 진술하고 있지만 그를 입증할 다른 증언 등 보강증거가 없어 이 역시 범죄의 증명이 없는 경우에 해당한다"며 B씨 등과 '
 '마찬가지로 무죄를 선고했다. 이와 관련 검찰 관계자는 "판결문에 적시된 무죄 이유를 검토한 뒤 항소여부를 결정하겠다"고 말했다. '
 'sun0701@')


In [45]:
lda_model[X_lda[idx]]

[(27, 0.69215721), (34, 0.013114556), (47, 0.16727021), (48, 0.010923133)]

In [46]:
lda_vec = np.zeros((len(textInfo), ntopics))
lda_vec.shape

(4722, 50)

In [47]:
for i in range(len(lda_vec)):
    idx = [x[0] for x in lda_model[X_lda[i]]]
    prob = [x[1] for x in lda_model[X_lda[i]]]
    lda_vec[i][idx] = prob

ValueError: shape mismatch: value array of shape (7,) could not be broadcast to indexing result of shape (8,)

In [ ]:
# raise small number to calculate cosine distances
lda_vec = lda_vec + 0.0001

In [ ]:
X_l2v = np.hstack((X_d2v, lda_vec))

In [ ]:
cnum = 10

In [ ]:
km = KMeans(n_clusters=cnum, init='k-means++', max_iter=300, n_init=20, tol=0.0001)
km.fit(X_l2v)
print("Silhouette Coefficient: %0.3f" % silhouette_score(X_l2v, km.labels_))

In [ ]:
dfRes = pd.DataFrame(textInfo, columns=['text'])
dfRes['label'] = km.labels_

for i in range(cnum):
    print('label{}: {}'.format(i, len(dfRes[dfRes.label == i])))

In [ ]:
for i in range(cnum):
    print('cluster {}:'.format(i)),
    print_topwords(df=dfRes.loc[dfRes['label'] == i], num=5, bar=False)
    print("")